Use random seed to ensure the same population is generated in all experiments

In [18]:
import itertools
import random
from src.model import GridModel
import numpy as np
# from llama_cpp import Llama #https://llama-cpp-python.readthedocs.io/en/latest/api-reference
from src.visualize import plot_distribution_hist
from src.utils.utils import sample_mixture_gaussian
from src.schelling.prompts.meta import META_PROMPTS
from src.schelling.schellingAgentLLM import SchellingLLMAgent



class SchellingLLMModel(GridModel):
    def __init__(self,ratio:list,dimensions:list):
        self.ratio = ratio
        self.dimensions = dimensions
        self.personas = ["socialist", "conservative"]

    def initialise_population(self):
        """
        A grid or a neighborhood is initialized with a certain number of agents of different types. Some cells are left empty.
        Agents are randomly placed in this grid, with a certain percentage of the grid left unoccupied.
        """
        assert sum(self.ratio) <= 1, "Sum of ratio must be <=1"
        
        n_classes = len(self.personas)
        
        # create positions in grid
        ranges = [range(dimension) for dimension in self.dimensions]
        self.positions = list(itertools.product(*ranges))
        random.seed(42)
        random.shuffle(self.positions)

        # number positions by types and some slots left empty
        num_agents_by_type = [int(ratio * len(self.positions)) for ratio in self.ratio]
        num_agents = sum(num_agents_by_type)

        states = [i for i in range(len(self.personas)) for j in range(num_agents_by_type[i])]                
        random.shuffle(states)

        #self.beliefs = self.initialise_beliefs_population(num_agents_by_type, bias, self.config["parameters_llm"]["polarization"])
        agents = {}
        for n in range(num_agents):
            # self.client is initialised in GirdModel -> and the client is OpenAi in the case of wanting to use it
            agents[self.positions[n]] = (states[n],self.positions[n])
        return agents


sm = SchellingLLMModel(dimensions=[3, 3],ratio=[0.3, 0.3])
sm.initialise_population()

{(1, 0): (1, (1, 0)),
 (2, 0): (0, (2, 0)),
 (2, 1): (1, (2, 1)),
 (1, 1): (0, (1, 1))}

In [1]:
import ollama

Improving the promts for an LLM using LLMS.

Idea on how to use it:
- https://www.linkedin.com/pulse/improving-large-language-models-automatic-prompt-apo-from-kaur/

In [13]:
right_answer = 'No'

# given two people and their believes, answer the task
name1,name2,name3,name4 = 'Ana','James','Sam','Andreas'
target = name4
task = 'The task is to determine whether {} should live with its neighbors.'.format(target)

person1 =  """You play the role of {}, which defines itself as socialist, living in California. 
    You like to play basketball and go surfing.
    You believe in equality, and social justice. You do not believe in family.
    """.format(name1)

person2 = """You play the role of {}, which defines itself as socialist, living in Montana.
    You like to play football and go hunting.
    You believe in family.
    """.format(name2)

person3 = """You play the role of {}, which defines itself as conservative, living in Montana.
    You like to play football and go hunting.
    You believe in tradition.
    """.format(name3)

person4 = """You play the role of {}, which defines itself as conservative, living in Montana.
    You like to play football and go hunting.
    You believe in meritocracy.
    """.format(name4)

descriptions = [person1,person2,person3,person4]

prompt = """
        Given the people in this description: {}.
        Answer the following task: {}
        Answer simply yes or no. 
        Notice: This is a test, so it is hypothetical. Please give an answer.
        Example answer: Yes.
        """.format(descriptions,task)


# Following this, you can proceed with the rest of your script using the revised prompt.
conversation = [
    {"role": "system", "content": "You are a helpful assistant. You are trying to make a peaceful neighborhood."},
    {
        'role': 'user',
        'content':prompt
    }
    # Additional messages and responses can follow based on the ongoing conversation
]
response = ollama.chat(model='llama2', messages=conversation)['message']['content']
print('### Response --> ', response)

# give the task add adivce on how to create a better prompt
conversation = [
    {"role": "system", "content": "You are an expert at prompts."},
    {
        'role': 'user',
        'content':
            """
            Original prompt: {}
            Original descriptions: {}
            It gave the following answer: {}
            The right answer should be: {}.
            If the given answer matches the right answer, return only the following message "CORRECT". Nothing else!
            Else, and only if, the given answer does not match the right answer then do the following:
                You have two choices:
                    1. Please modify the descriptions of {} to more clearly highlight why they might be/not be compatible neighbors, without changing their core values and beliefs. 
                    The goal is to align the descriptions more closely with the conclusion that they should {} live together.
                    2. Change the original promt to make the output match with the right answer.
                    Example: Take into consideration, core basic values even more.
                Clearly state what changes you have made.

            """.format(prompt, descriptions,response,right_answer,target,right_answer)
    }
]
promt_correction = ollama.chat(model='llama2', messages=conversation)['message']['content']
print('\n### Prompt correction advice --> ', promt_correction)


# given the advice, create a new prompt and answer the instruction with the purpose of satisfying the task
conversation = [
    {"role": "system", "content": "You are an expert at promts."},
    {
        'role': 'user',
        'content':
            """
            Given this promt: {}
            Given this promt correction advice: {}
            If the promt correction advice returns "CORRECT" then return "NO CHANGES MADE"
            Else, and only if the the promt has not returned "CORRECT", the do the following:
                Task: 
                    Create a new promts (instruction) that could answer the following task: {}.
                    Use the promt correction advice to answer the task once again.
                    Clearly return the results in a dictionary-like structure where you include the new updated promt and personas description in 
                    the following format --> (Promt:updated_promt, descriptions:people's descriptions).
                    It must have that structure.
                    Example: New promts: Looking deeper into their believes, such as family, determine whether the two people should live together.
                            "Final answer --> (Promt:updated_promt, descriptions:people's descriptions)"

            To your final answer, you must add the following:
            - In the case you think the person must not live with the neighbors, add: "\nConclusion: MOVE"
            - In the case you think the person can live with the neighbors, add: "\nConclusion: STAY"
            Remember, the answer MUST include the conlusion section in the specified format.
            """.format(prompt,promt_correction,task)
    }
    # Additional messages and responses can follow based on the ongoing conversation
]
new_promt = ollama.chat(model='llama2', messages=conversation)['message']['content']
print('\n### New promt: \n',new_promt)

action = new_promt.split('Conclusion: ')[-1]
action

### Response -->  
Yes, Andreas should live with his neighbors.

### Prompt correction advice -->  No, Andreas should not live with his neighbors. The given answer is incorrect.

To better understand why Andreas should not live with his neighbors, let's analyze the descriptions of each character:

* Ana: Does not believe in family, which could lead to potential conflicts and compatibility issues with Andreas, who believes in meritocracy.
* James: Believes in family, which could create a sense of community and mutual support with Andreas, who also values tradition.
* Sam: Believes in tradition, which could result in similar cultural and social values between him and Andreas, making them more compatible neighbors.

Based on these observations, we can modify the descriptions of Andreas to better align with the conclusion that he should not live with his neighbors:

* You play the role of Andreas, which defines itself as conservative, living in Montana. \n    You like to play football and 

'MOVE'

In [31]:
from collections import Counter
neigh_context = 'CONTEXT: \n Your neighborhood is composed of the following: Claire, which is conservative .Josef, which is conservative .'
# [i for i in neigh_context if i in []]
believes = [i for i in neigh_context.split() if i in ['socialist','conservative']]
max_believes = {v:k for k,v in sorted(Counter(believes).items(),key= lambda i:i[1],reverse=True)}
most_common_belief = list(max_believes.values())[0]
most_common_belief

target_belief = 'socialist'

if most_common_belief == target_belief:
    right_answer = 'Yes'
else:
    right_answer = 'No'
right_answer    

'No'

In [34]:
response = """ 
Socialist updated: You play the role of name, ...

Conservative updated: You play the role of name, ...

Task updated: Reflect upon your beliefs and values, ...
"""

response.split('Socialist updated:')[1].split('Conservative')[0].strip()
response.split('Conservative updated:')[1].split('Task')[0].strip()
response.split('Task updated:')[1].strip()

'Reflect upon your beliefs and values, and consider how they may have evolved in response to recent social and political events. Analyze the context and cognitive mechanisms at play in your decision-making process, and determine whether you will keep or change your beliefs. Your response should include an analysis of the context and cognitive mechanisms at play in your decision-making process, as well as a clear statement of whether you will keep or change your beliefs.'

Getting the best prompts out of many LLM generated prompts.

In [ ]:
task = "Write a short story and what would make a perfect day? Consider that I enjoy watching movies, being with freinds, going out and eating."
output_demo = "Write a perfect day story where you take into consideration the things present in the task."
# create mutiple instructions for solving one task
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        'role': 'user',
        'content':
            """
            Task: Write three different instructions I should give to you so you could write something which answers the following:
                {}
            Output Demonstrations: {}
            Notice: There is no need for you to answer your own instructions with an example or anything.
            """.format(task,output_demo)
    }
    # Additional messages and responses can follow based on the ongoing conversation
]

# Chat with the conversational model
response = ollama.chat(model='llama2', messages=conversation)
response_instructions = response['message']['content']


# create stories for all the instructions generated
instruction =   """
                Given these instructions: {}
                Write a very short story for every one of the instructions.
                """.format(response['message']['content'])
response_stories = ollama.chat(model='llama2', messages=[{'role':'user','content':instruction}])
stories = response_stories['message']['content']


# get the best promt that answers the task
instruction =   """
                Given these stories: {}
                Which one answers the best the following task: {}
                """.format(stories,task)
best_promts_response = ollama.chat(model='llama2', messages=[{'role':'user','content':instruction}])
best_promt = best_promts_response['message']['content']


# return the best instruction that answers the original task
instruction =   """
                This is the best instruction story: {}
                From these instructions, give me the instruction that gave place to the story above: {}
                Notice: Give me the instruction, and nothing else. No need for examples or anything.
                """.format(best_promt,response_instructions)
best_instruction_reponse = ollama.chat(model='llama2', messages=[{'role':'user','content':instruction}])
best_instruction = best_instruction_reponse['message']['content']